In [1]:
'''
带有随机失活功能的前馈神经网络的PaddlePaddle实践代码。
'''
import paddle
from paddle import nn, optimizer, metric


#设定超参数。
INPUT_SIZE = 784 
HIDDEN_SIZE = 256 
NUM_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
DROPOUT_RATE = 0.2

# 搭建前馈神经网络。
paddle_model = nn.Sequential(
    
    #添加有256个神经元的隐藏层。
    nn.Linear(INPUT_SIZE, HIDDEN_SIZE),
    
    #设定激活函数为ReLU。
    nn.ReLU(),
    
    #添加随机失活层。
    nn.Dropout(DROPOUT_RATE),
    
    #添加有10个神经元的输出层。
    nn.Linear(HIDDEN_SIZE, NUM_CLASSES)
)

#初始化前馈神经网络模型。
model = paddle.Model(paddle_model)

# 为模型训练做准备，设置优化器，损失函数和评估指标。
model.prepare(optimizer=optimizer.Adam(learning_rate=LEARNING_RATE, parameters=model.parameters()), 
              loss=nn.CrossEntropyLoss(),
              metrics=metric.Accuracy())

sysctl: unknown oid 'machdep.cpu.leaf7_features'


In [2]:
import pandas as pd


#使用pandas，读取fashion_mnist的训练和测试数据文件。
train_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_train.csv')
test_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_test.csv')

#从训练数据中，拆解出训练特征和类别标签。
X_train = train_data[train_data.columns[1:]]
y_train = train_data['label']

#从测试数据中，拆解出测试特征和类别标签。
X_test = test_data[train_data.columns[1:]]
y_test = test_data['label']

In [3]:
from sklearn.preprocessing import StandardScaler


#初始化数据标准化处理器。
ss = StandardScaler()

#标准化训练数据特征。
X_train = ss.fit_transform(X_train)

#标准化测试数据特征。
X_test = ss.transform(X_test)

In [4]:
from paddle.io import TensorDataset


X_train = paddle.to_tensor(X_train.astype('float32'))
y_train = y_train.values

#构建适用于PaddlePaddle模型训练的数据集。
train_dataset = TensorDataset([X_train, y_train])

# 启动模型训练，指定训练数据集，设置训练轮次，设置每次数据集计算的批次大小。
model.fit(train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step  40/938 [>.............................] - loss: 0.5924 - acc: 0.6953 - ETA: 4s - 5ms/step

/opt/anaconda3/envs/python_paddle/lib/python3.8/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 938/938 [==============================] - loss: 0.3881 - acc: 0.8282 - 5ms/step          
Epoch 2/5
step 938/938 [==============================] - loss: 0.3205 - acc: 0.8697 - 5ms/step          
Epoch 3/5
step 938/938 [==============================] - loss: 0.1736 - acc: 0.8822 - 5ms/step          
Epoch 4/5
step 938/938 [==============================] - loss: 0.1645 - acc: 0.8892 - 5ms/step          
Epoch 5/5
step 938/938 [==============================] - loss: 0.1823 - acc: 0.8960 - 5ms/step          


In [5]:
X_test = paddle.to_tensor(X_test.astype('float32'))
y_test = y_test.values

#构建适用于PaddlePaddle模型测试的数据集。
test_dataset = TensorDataset([X_test, y_test])

#启动模型测试，指定测试数据集。
result = model.evaluate(test_dataset, verbose=0)

print('带有随机失活功能的前馈神经网络（PaddlePaddle版本）在fashion_mnist测试集上的准确率为: %.2f%%。' %(result['acc'] * 100))

带有随机失活功能的前馈神经网络（PaddlePaddle版本）在fashion_mnist测试集上的准确率为: 88.92%。
